In [2]:
from EnKF_func import *

In [3]:
EnKF(SAVEDATA=3,data_dir='exp_dt2.npz')

(2.2030595249606093,
 2.1747934086653076,
 0.7208405539502045,
 0.5310033013042192,
 2.9290817516107803,
 3.0247897700112296)

In [4]:
dt2 = np.load('exp_dt2.npz')

In [5]:
dt2.files

['DESC',
 'RSEED',
 'HTYPE',
 'HERROR',
 'LINEAR',
 'FORECAST_ERROR',
 'INFLATION',
 'ndim',
 'pars',
 'nobs',
 'deltaobs',
 'dobs',
 'Hmat',
 'sigmaobs',
 'infl_lin',
 'infl_nlin',
 'sigmainit',
 'nens',
 'ferr',
 'truestate',
 'trueobs',
 'yobs',
 'xfm',
 'xfcov',
 'xam',
 'xacov',
 'yfm',
 'yfcov',
 'xferravgxt',
 'xaerravgxt',
 'xferr10avgxt',
 'xaerr10avgxt',
 'xferr30avgxt',
 'xaerr30avgxt']

In [6]:
truestate = dt2['truestate']

In [7]:
truestate.shape

(40, 51)